In [2]:
# Importing the dependancies
import datetime
import pyttsx3
import speech_recognition as sr
import wikipedia
import webbrowser
import os
import random
import json
import pickle
import numpy as np
from urllib.parse import urlparse

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import load_model
import keyboard

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import warnings
warnings.filterwarnings("ignore")
from googlesearch import search
import pywhatkit as kt
from bs4 import BeautifulSoup
import requests

Following program is just for normal conversation..

In [3]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())

In [4]:
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbot.h1')

In [5]:
# Function to clean up the sentence
def clean_up_sentence(sentence):
    
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [7]:
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details = True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return (np.array(bag))

In [8]:
# Function to predict the target class
def predict_class(sentence, model):
    
    # filter out predictions below a threshold
    p = bow(sentence, words, show_details = False)
    res = model.predict(np.array([p]))[0]
    error = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > error]
    
    # sort by strength of probability
    results.sort(key = lambda x:x[1], reverse = True)
    return_list = []
    
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [9]:
# function to get the response from the model

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag'] == tag):
            result = random.choice(i['responses'])
            break
    return result

# function to predict the class and get the response

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

Now From here the Jarvis Assistant code starts

In [10]:
# For giving the voice to our jarvis..
engine = pyttsx3.init('sapi5')
voices = engine.getProperty('voices')
# If you want to have a male voice..keep 0 there else change it to 1 for a girl voice
engine.setProperty('voice', voices[0].id)

In [11]:
def speak(audio):
    '''
        args: any string
        working: It will just speak anything you wrote in that audio argument
    '''
    engine.say(audio)
    engine.runAndWait()

In [12]:
def wishme():
    '''
        working: Will look for the time and wish you accordingly
                 It will also give you some intro about itself.
    '''
    hour = int(datetime.datetime.now().hour)
    if hour >= 0 and hour < 12:
        speak("Good Morning!")
    elif hour >= 12 and hour < 18:
        speak("Good Afternoon!")
    else:
        speak("Good Evening!")
    
    speak("Jarvis Here.")
    speak("Your AI assistant. Please tell how may I help you!")

In [13]:
def takeCommand():
    '''
        It takes microphone input from the user and returns string output
    '''
    
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        r.pause_threshold = 1
        audio = r.listen(source)
    
    try:
        print("Recognizing...")
        query = r.recognize_google(audio, language='en-In')
        print(f"User said: {query}\n")
    
    except Exception as e:
        print("Say that again please...")
        speak("Say that again please")
        return "None"
    return query

In [14]:
def talkNormal(work):
    speak(chatbot_response(work))
    print("-"*50)

In [15]:
def googleSearch(ques):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get("https://www.google.com/")
    driver.find_element(By.XPATH, "//input[contains(@class, 'gLFyf gsfi')]").send_keys(ques)
    keyboard.press('enter')
    keyboard.release('enter')
    urls = []
    new_url = []
    for j in search(ques, num = 30, stop = 30, lang = 'en',  pause = 2):
        domain = urlparse(j).netloc.split('.')
        if domain[1] not in new_url:
            new_url.append(domain[1])
            urls.append(j)

In [16]:
if __name__ == "__main__":
    wishme()
    # query = takeCommand().lower()
    url_num = 0
    urls = []
    new_url = []
    # try:
    #     query = query.replace('jarvis', "")
    # except:
    #     pass
    # talkNormal(query)
    while True:
        # Taking the response
        if keyboard.is_pressed("e"):
            query = takeCommand().lower()
            try:
                query = query.replace('jarvis', "")
            except:
                pass
            # if 'you' or 'your' or 'yours' in query:
            #     talkNormal(query)
            if 'site' or 'website' in query:
                driver = webdriver.Chrome(ChromeDriverManager().install())
                if 'open next' in query:
                    url_num += 1
                    try:
                        speak("Hold on till I fetch your results..")
                        driver.get(urls[url_num])
                        speak("Here you go!")
                    except:
                        url_num -= 1
                        speak("Looks like you been playing with me..Well Search Engines don't allow too many websites to me. These were some of the best ones")
                elif 'open previous' in query:
                    url_num -= 1
                    try:
                        speak("Fetching your results user..just one second")
                        driver.get(urls[url_num])
                        speak("Yeah now go ahead")
                    except:
                        url_num += 1
                        speak("Well..This was the best result user. There is no previous website you can move to")
                elif 'open' in query:
                    try:
                        speak("Opening the ")
                        driver.get(urls[url_num])
                    except:
                        speak("Looks like you just messed here..Probably no google search executed by you")
                        speak("Try Speaking something which I can search upon web")
            elif 'search' or 'google' or 'what' or 'look for' in query:
                speak("Searching for your results")
                if 'search' in query:
                    query = query.replace('search', "")
                if 'google' in query:
                    query = query.replace('google', "")
                if 'look for' in query:
                    query = query.replace('look for', "")
                url_num = 0
                googleSearch(query)
            
        elif keyboard.is_pressed("q"):
            speak("Have a Nice Day Sir!")
            break

Listening...
Recognizing...
User said: Google Shahrukh Khan



[WDM] - Downloading: 100%|██████████| 6.29M/6.29M [00:19<00:00, 344kB/s] 
